In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv('NLP_dataset.csv', encoding = "ISO-8859-1")
df = df[:100000]
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Saiprasad,NNP,Person
1,NaN,23,CD,O
2,NaN,years,NNS,O
3,NaN,Male,JJ,O
4,NaN,Capsule,NNP,Medicine


In [3]:
df.isnull().sum()

Sentence #    210
Word            0
POS             0
Tag             0
dtype: int64

In [4]:
df = df.fillna(method='ffill')
df['Sentence #'].nunique(), df.Word.nunique(), df.Tag.nunique()

(6, 80, 5)

In [5]:
df.groupby('Tag').size().reset_index(name='counts')

,Tag,counts
0,Dose,14
1,Frequency,43
2,Medicine,35
3,O,113
4,Person,11


In [6]:
X = df.drop('Tag', axis=1)
v = DictVectorizer(sparse=False)
X = v.fit_transform(X.to_dict('records'))
y = df.Tag.values
classes = np.unique(y)
classes = classes.tolist()

In [7]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter

In [8]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
getter = SentenceGetter(df)
sentences = getter.sentences

In [9]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
    return features
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, postag, label in sent]
def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [10]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [11]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [12]:
new_classes = classes.copy()
new_classes.remove('O')
new_classes

['Dose', 'Frequency', 'Medicine', 'Person']

In [13]:
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred, labels = new_classes))
print("Accuracy: %f"%(sklearn_crfsuite.metrics.flat_accuracy_score(y_test, y_pred)))

             precision    recall  f1-score   support

       Dose       1.00      0.75      0.86         4
  Frequency       0.89      1.00      0.94         8
   Medicine       1.00      0.75      0.86         8
     Person       1.00      1.00      1.00         4

avg / total       0.96      0.88      0.91        24

Accuracy: 0.921569


In [14]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))
print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(5))
print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-5:])

Top likely transitions:
Frequency -> O       1.913464
Medicine -> Medicine 1.636850
Medicine -> Dose    1.231886
Person -> Person  1.052644
Dose   -> Frequency 0.836473

Top unlikely transitions:
Person -> Medicine -0.513618
Person -> Frequency -0.726086
Medicine -> Person  -0.798690
O      -> Person  -0.993984
Medicine -> O       -1.538591


In [15]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))
print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(10))
print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-10:])

Top positive:
1.687348 Dose     word[-2:]:mg
1.609308 Frequency word.isdigit()
1.228623 Person   word.istitle()
1.192162 O        +1:postag[:2]:NN
1.145563 Person   BOS
1.143241 O        -1:postag[:2]:NN
1.027509 Medicine postag:NNP
1.024770 Person   postag:NNP
1.004573 O        +1:word.lower():years
1.004077 O        -1:postag:NNS

Top negative:
-0.288096 O        +1:postag:CD
-0.288096 O        +1:postag[:2]:CD
-0.312406 Dose     bias
-0.320021 Frequency +1:postag:IN
-0.320021 Frequency +1:postag[:2]:IN
-0.324014 Frequency +1:word.istitle()
-0.346968 Frequency -1:postag:NNP
-0.420040 O        word.istitle()
-0.571004 Frequency -1:word.istitle()
-0.862610 O        postag:NNP


In [16]:
import eli5
eli5.show_weights(crf)

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [17]:
eli5.show_weights(crf,top=10, feature_re='^word\.is',
                  horizontal_layout=False, show=['targets'])

Weight?,Feature
-0.099,word.istitle()
-0.221,word.isdigit()
Weight?,Feature
+1.609,word.isdigit()
-0.046,word.istitle()
Weight?,Feature
+0.362,word.isupper()
Weight?,Feature
-0.092,word.isdigit()
-0.420,word.istitle()
